<a href="https://colab.research.google.com/github/allisonpg12/analitica3proyecto3/blob/main/CASO_SALUD_APG_AVRQ_MIVZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CASO DE ESTUDIO SALUD**

## **Importar librerias**

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, RidgeCV, Lasso
from functools import reduce
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.feature_selection import SelectFromModel

%matplotlib inline
from sklearn import tree
from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
import sqlite3
from datetime import datetime

# Evitar salidas de Warnings
import warnings
warnings.filterwarnings("ignore")

## **Lectura de datos - Procesamiento de bases**

In [ ]:
#conectar el notebook al drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
